# Semantic Kernel s integrací serveru OpenBnB MCP

Tento notebook ukazuje, jak použít Semantic Kernel se skutečným serverem OpenBnB MCP pro vyhledávání skutečných ubytování na Airbnb pomocí MCPStdioPlugin. Pro přístup k LLM využívá Microsoft Foundry. Pro nastavení svých proměnných prostředí můžete sledovat [Setup Lesson ](/00-course-setup/README.md)


## Importujte potřebné balíčky


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Vytvoření připojení pluginu MCP

Připojíme se k [OpenBnB MCP serveru](https://github.com/openbnb-org/mcp-server-airbnb) pomocí MCPStdioPlugin. Tento server poskytuje funkci vyhledávání Airbnb prostřednictvím balíčku @openbnb/mcp-server-airbnb.


## Vytvoření klienta

V tomto příkladu použijeme Microsoft Foundry pro přístup k našemu LLM. Ujistěte se, že máte správně nastavené proměnné prostředí.


## Konfigurace prostředí

Nakonfigurujte nastavení Azure OpenAI. Ujistěte se, že máte nastavené následující proměnné prostředí:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Porozumění integraci OpenBnB MCP

Tento notebook se připojuje k **skutečnému OpenBnB MCP serveru**, který poskytuje reálnou funkčnost vyhledávání Airbnb.

### Jak to funguje:

1. **MCPStdioPlugin**: Používá standardní vstupní/výstupní komunikaci se serverem MCP
2. **Skutečný NPM balíček**: Stáhne a spustí `@openbnb/mcp-server-airbnb` přes npx
3. **Živá data**: Vrací skutečná data nemovitostí Airbnb z jejich API
4. **Objevování funkcí**: Agent automaticky zjišťuje dostupné funkce ze serveru MCP

### Dostupné funkce:

Server OpenBnB MCP obvykle poskytuje:
- **search_listings** - Vyhledávání nemovitostí Airbnb podle lokaci a kritérií
- **get_listing_details** - Získání podrobných informací o konkrétních nemovitostech
- **check_availability** - Kontrola dostupnosti na konkrétní data
- **get_reviews** - Získávání recenzí nemovitostí
- **get_host_info** - Získání informací o hostitelích nemovitostí

### Požadavky:

- Nainstalovaný **Node.js** ve vašem systému
- **Internetové připojení** ke stažení balíčku MCP serveru
- Dostupnost **NPX** (součást Node.js)

### Testování připojení:

Můžete ručně otestovat server MCP spuštěním:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Toto stáhne a spustí OpenBnB MCP server, ke kterému se poté Semantic Kernel připojí pro získání reálných dat Airbnb.


## Spuštění agenta s OpenBnB MCP serverem

Nyní spustíme AI agenta, který se připojuje k serveru OpenBnB MCP, aby vyhledal skutečné ubytování Airbnb ve Stockholmu pro 2 dospělé a 1 dítě. Klidně změňte seznam `user_inputs` pro úpravu kritérií vyhledávání.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Shrnutí
Gratulujeme! Úspěšně jste vytvořili AI agenta, který se integruje s vyhledáváním ubytování v reálném světě pomocí Model Context Protocol (MCP):

## Použité technologie:
- Semantic Kernel - Pro vytváření inteligentních agentů s Azure OpenAI
- Microsoft Foundry - Pro schopnosti LLM a dokončování chatů
- MCP (Model Context Protocol) - Pro standardizovanou integraci nástrojů
- OpenBnB MCP Server - Pro skutečnou funkci hledání na Airbnb
- Node.js/NPX - Pro spuštění externího MCP serveru

## Co jste se naučili:
- Integrace MCP: Připojení agentů Semantic Kernel k externím MCP serverům
- Přístup k datům v reálném čase: Vyhledávání skutečných nemovitostí na Airbnb přes živá API
- Komunikace protokolem: Používání stdio komunikace mezi agentem a MCP serverem
- Objevování funkcí: Automatické zjišťování dostupných funkcí z MCP serverů
- Streaming odpovědí: Zachytávání a logování volání funkcí v reálném čase
- HTML vykreslování: Formátování odpovědí agenta se stylizovanými tabulkami a interaktivními zobrazeními

## Další kroky:
- Integrace dalších MCP serverů (počasí, lety, restaurace)
- Vytvoření multi-agentního systému kombinujícího MCP a A2A protokoly
- Vytvoření vlastních MCP serverů pro vaše zdroje dat
- Implementace persistentní paměti konverzace přes sezení
- Nasazení agenta do Azure Functions s orchestrací MCP serveru
- Přidání uživatelské autentizace a možností rezervací


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Prohlášení o vyloučení odpovědnosti**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože usilujeme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho mateřském jazyce by měl být považován za autoritativní zdroj. Pro kritické informace se doporučuje profesionální lidský překlad. Nejsme odpovědní za jakákoliv nedorozumění nebo chybné interpretace vyplývající z použití tohoto překladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
